## **Сервисные функции**

In [ ]:
import os
import shutil

# @title Функция удаления всех папок и файлов в directory
def delete_all_files_and_folders(directory):
    try:
        # Переход в указанную директорию
        os.chdir(directory)

        # Удаление всех файлов и папок
        for file in os.listdir():
            file_path = os.path.join(directory, file)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")

        print(f"All files and folders in {directory} have been deleted.")
    except Exception as e:
        print(f"Failed to delete files and folders. Reason: {e}")


## **Предварительные действия**

In [ ]:
!nvidia-smi


Sat Dec 23 12:16:23 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)


/content


In [ ]:
# @title Удаление всех папок и файлов в root_path
delete_all_files_and_folders(HOME)


All files and folders in /content have been deleted.


## **Установка YOLOv8**

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image


## **Обнаружение объектов**

In [ ]:
# Монтируем Google Диск
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil
import datetime

# @title Определение путей для использования лучшей модели best.pt с Google Disk
# Базовый путь, где хранится модель и др.
base_path = '/content/drive/MyDrive/Intership_03/YOLOv8_trained/'

last_best_path = f"{base_path}Last_Best/"
test_predict_path = f"{base_path}Test_Predict/"

print(f'Путь, где сохранена модель: {last_best_path}')


Путь, где сохранена модель: /content/drive/MyDrive/Intership_03/YOLOv8_trained/Last_Best/


In [ ]:
# @title Обнаружение объектов (detection), используя сохранённую лучшую модель best.pt

min_conf = 0.5

!yolo task=detect mode=predict model={last_best_path}best.pt source='{test_predict_path}' conf={min_conf} save=True


Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs

WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/9 /content/drive/MyDrive/Intership_03/YOLOv8_trained/Test_Predict/vid2_03m_39s_original.png: 384x640 3 crane-autos, 6 crane-towers, 4 tractors, 1 truck-dump, 2 truck-freight-cars, 224.6ms
image 2/9 /content/drive/MyDrive/Intership_03/YOLOv8_trained/Test_Predict/vid2_03m_49s_original.png: 384x640 2 crane-autos, 4 crane-towers, 2 tractors, 1 truck, 2 truck-freight-cars, 25.2ms
image 3/9 /content/drive/MyDrive/Intership_03/YOLOv8_trained/Test_Predict/vid2_03m_59s_original.png: 384x640 3 crane-autos, 6 crane-towers, 2 tractors, 1 tractor-manipulator, 1 truck, 1 truck-freight-car, 25.2ms
image 4/9 /content/drive/MyDrive/Intership_03/YOLOv8_trained/Test_Predict/vid2_04m_09s_original.png: 384x640 3 crane-autos, 5 crane-towers, 1 tractor, 1 truck, 1 truck-freight-car, 25.1ms
im

In [ ]:
import glob
from IPython.display import Image, display

# @title Вывод на экран результата обнаружения строительной техники

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.png')[:]:
      display(Image(filename=image_path, width=1500))
      print("\n")


Output hidden; open in https://colab.research.google.com to view.

## **Вывод:**

**Качеством обнаружения строительной техники удовлетворён 🌝**

При conf=0.5 нет ошибочного обнаружения класса "truck" (conf=0.35), но также и не обнаружен один объект класса "tractor" (conf=0.32)

В принципе этот объект класса "tractor" (conf=0.32) достаточно мелкий и невзрачный, сливающийся с фоном.

**P.S.:**

Количество размеченных изображений: **200**

Для до-обучения YOLOv8 была выбрана модель: **yolov8m.pt**

epochs: выбрано 300, закончилось на 213, т.к. на протяжении 50 эпох не было улучшений (т.е. по факту **обучение 163 эпохи**)


По результатам валидации обученной модели YOLOv8 можно сделать следующие выводы:

**1.	Общая оценка (All):**

>	Количество изображений: 40

>	Общее количество объектов: 164

>	Precision (P): 83.6%

>	Recall (R): 74.3%

>	mAP50 (Mean Average Precision): 82.7%

>	mAP50-95 (Mean Average Precision with IoU from 50% to 95%): 65.1%

**2.	Скорость:**

>	Время предобработки: 0.2 мс

>	Время инференса: 12.4 мс

>	Время потери (loss): 0.0 мс

>	Время постобработки: 3.0 мс

В общем, **модель показывает хорошие результаты с высокими значениями точности и полноты для большинства классов объектов**, хотя датасет не сбалансирован и можно применить аугментацию к уже размеченной базе. Либо ещё разметить, т.к. всего изображений в базе 757 (из них размечено только 200). Ещё в запасе есть 557 изображений.

Также, **согласно mAP50 и mAP50-95, общую производительность модели можно оценить как хорошую.**